In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="4" 

In [2]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable

from lib.utils.config_parse import cfg_from_file
from lib.ssds_train import test_model, train_model

from lib.dataset.dataset_factory import load_data
from lib.utils.config_parse import cfg
from lib.modeling.model_builder import create_model

from lib.layers import Detect
from lib.utils.timer import Timer

import sys
import pickle
import numpy as np



config_file =  './experiments/cfgs/ssd_resnet18_train_voc.yml'
cfg_from_file(config_file)

print(config_file)

./experiments/cfgs/ssd_resnet18_train_voc.yml


In [2]:
 # Load data
print('===> Loading data')
train_loader = load_data(cfg.DATASET, 'train') if 'train' in cfg.PHASE else None
eval_loader = load_data(cfg.DATASET, 'eval') if 'eval' in cfg.PHASE else None
test_loader = load_data(cfg.DATASET, 'test') if 'test' in cfg.PHASE else None

===> Loading data


In [3]:
cfg.MODEL

{'NETS': 'resnet_18',
 'SSDS': 'ssd',
 'HALF_PRECISION': True,
 'IMAGE_SIZE': [300, 300],
 'NUM_CLASSES': 21,
 'FEATURE_LAYER': [[7, 9, 'S', 'S', '', ''], [128, 256, 512, 256, 256, 256]],
 'STEPS': [[8, 8], [16, 16], [32, 32], [64, 64], [100, 100], [300, 300]],
 'SIZES': [[30, 30],
  [60, 60],
  [111, 111],
  [162, 162],
  [213, 213],
  [264, 264],
  [315, 315]],
 'ASPECT_RATIOS': [[1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2], [1, 2]],
 'CLIP': True,
 'NUM_FUSED': 3}

In [5]:
# Build model
print('===> Building model')
model, priorbox = create_model(cfg.MODEL)
with torch.no_grad():
    priors = Variable(priorbox.forward())
    detector = Detect(cfg.POST_PROCESS, priors)

===> Building model
==>Feature map size:
[(38, 38), (19, 19), (10, 10), (5, 5), (3, 3), (1, 1)]


In [5]:
model.base

ModuleList(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): _basicblock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (conv2): Conv2d(64, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (5): _basicblock(
    (conv1): Conv2d(256, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 